<a href="https://colab.research.google.com/github/arzharch/Chatbot-using-Spacy/blob/main/spacy_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy requests
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import random
import requests
import gradio as gr

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# Define a list of intents (topics the chatbot can handle)
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
        "responses": ["Hi there!", "Hello!", "Hey!", "Good to see you!"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "Goodbye", "See you later", "Farewell"],
        "responses": ["Goodbye!", "See you later!", "Take care!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thanks", "Thank you", "Appreciate it"],
        "responses": ["You're welcome!", "No problem!", "Glad I could help!"]
    },
    {
        "tag": "about",
        "patterns": ["What are you?", "Who are you?", "What can you do?"],
        "responses": ["I am an AI assistant for Google Colab. I can write code and answer questions."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like?", "Tell me the weather", "Weather update"],
        "responses": ["Let me check the weather for you..."]
    }
]

# Function to fetch real-time weather information from an API (e.g., OpenWeatherMap)
def fetch_weather():
    api_key = '1e8ada7974c33733127a83dac0fb96c3'  # will be disabled when not used
    city = 'Panjim'
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Construct the API URL
    complete_url = base_url + "appid=" + api_key + "&q=" + city

    response = requests.get(complete_url)  # Correct variable name
    if response.status_code != 404:
        data = response.json()
        weather_description = data['weather'][0]['description']
        temperature_kelvin = data['main']['temp']
        temperature_celsius = temperature_kelvin - 273.15  # Convert from Kelvin to Celsius
        return f"The weather in {city} is {weather_description} with a temperature of {temperature_celsius:.2f}°C."
    else:
        return "Sorry, I couldn't fetch the weather information right now."

# Function to process user input and find the best matching intent
def process_input(user_input):
    doc = nlp(user_input)
    max_similarity = 0
    matched_intent = None

    # Match user input to intents based on similarity
    for intent in intents:
        for pattern in intent["patterns"]:
            pattern_doc = nlp(pattern)
            similarity = doc.similarity(pattern_doc)
            if similarity > max_similarity:
                max_similarity = similarity
                matched_intent = intent

    # Handle weather-specific intent by calling the API
    if matched_intent and matched_intent["tag"] == "weather":
        response = fetch_weather()
    elif matched_intent:
        response = random.choice(matched_intent["responses"])
    else:
        response = "I'm not sure I understand. Can you rephrase?"

    return response

# Define a function to use in the Gradio interface
def chatbot_interface(user_input):
    return process_input(user_input)

# Create a Gradio interface
interface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your message here..."),
    outputs="text",
    title="AI Chatbot",
    description="A simple AI chatbot that can respond to greetings, farewells, and other general queries.",
    theme="default"
)

# Launch the Gradio interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://750dae1659924685d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
